In [12]:
import nltk
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.tag import brill, brill_trainer 
from nltk.tag import BrillTaggerTrainer, RegexpTagger
from nltk.tag.perceptron import PerceptronTagger
from nltk.tbl.template import Template
from nltk.tag.brill import Pos, Word
from nltk.corpus import brown
from nltk.tag import CRFTagger

In [ ]:
from nltk.corpus import brown
brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')

In [13]:
nltk.help.upenn_tagset('NN-TL')

No matching tags found.


In [59]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [58]:
nltk.help.upenn_tagset('BED')

No matching tags found.


http://www.nltk.org/api/nltk.tag.html

In [2]:
def export_to_csv(pos_tagged_text, filename):
    string_pos_text = [ nltk.tag.tuple2str(tup) for tup in pos_tagged_text]
    one_line = ' '.join(string_pos_text)
    wfp = open(filename, 'w')
    wfp.write(one_line)
    wfp.close()
    print("Exported Data to: {}".format(filename))

In [3]:
# Load data
filename = 'data/feedback_cs2012_1.txt'
with open(filename, 'r')as fp:
    all_content = fp.read()
text = nltk.word_tokenize(all_content)

In [4]:
# Default Tagger
pos_tagged_text = nltk.pos_tag(text)
export_to_csv(pos_tagged_text, 'data/default_tagger.csv')

Exported Data to: data/default_tagger.csv


In [5]:
# Prepare Test Data 
test_data = []
test_data.append(pos_tagged_text)

In [6]:
# Prepare Train data
train_data = brown.tagged_sents(categories='news')[:1000]

In [7]:
#PerceptronTagger
tagger2 = PerceptronTagger(load=False)
tagger2.train(train_data)
perceptron_tagged_data = tagger2.tag(text)
accuracy = tagger2.evaluate(test_data)
export_to_csv(tagger2.tag(text), 'data/perception_tagger.csv')
print("Perception Tagger Accuracy: {}".format(accuracy))

Exported Data to: data/perception_tagger.csv
Perception Tagger Accuracy: 0.5556510528577567


In [8]:
# http://www.nltk.org/api/nltk.tag.html?highlight=hmm#module-nltk.tag.brill_trainer
#RegexpTagger
baseline = RegexpTagger([(r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
(r'(The|the|A|a|An|an)$', 'AT'),   # articles
(r'.*able$', 'JJ'),                # adjectives
(r'.*ness$', 'NN'),                # nouns formed from adjectives
(r'.*ly$', 'RB'),                  # adverbs
(r'.*s$', 'NNS'),                  # plural nouns
(r'.*ing$', 'VBG'),                # gerunds
(r'.*ed$', 'VBD'),                 # past tense verbs
(r'.*', 'NN')                      # nouns (default)
])

regex_tagged_data = baseline.tag(text)
baseline.evaluate(test_data) 

export_to_csv(regex_tagged_data, 'data/regrex_tagger.csv')
print("Regrex Tagger Accuracy: {}".format(accuracy))

Exported Data to: data/regrex_tagger.csv
Regrex Tagger Accuracy: 0.5556510528577567


In [60]:
# http://www.nltk.org/api/nltk.tag.html?highlight=hmm#module-nltk.tag.brill_trainer
#BrillTaggerTrainer
templates = [Template(Pos([-1])), Template(Pos([-1]), Word([0]))]
tt = BrillTaggerTrainer(baseline, templates, trace=3)
tagger1 = tt.train(brown_tagged_sents, max_rules=10)
brill_tagged_data = tagger1.tag(text)
accuracy = tagger1.evaluate(brown_tagged_sents) 

export_to_csv(tagger1.tag(text), 'data/brill_tagger.csv')
print("Brill Tagger Accuracy: {}".format(accuracy))

TBL train (fast) (seqs: 4623; tokens: 100554; tpls: 2; min score: 2; min acc: None)
Finding initial useful rules...
    Found 13404 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
37283728   0  46  | NN->, if Pos:NN@[-1] & Word:,@[0]
28582858   0   9  | NN->. if Pos:NN@[-1] & Word:.@[0]
20852085   0 124  | NN->IN if Pos:NN@[-1] & Word:of@[0]
12941294   0  23  | NN->CC if Pos:NN@[-1] & Word:and@[0]
11271127   0  38  | NN->IN if Pos:NN@[-1] & Word:in@[0]
 881 881   0   9  | NN->, if Pos:NNS@[-1] & Word:,@[0]
 750 750   0   7  | NN->. if Pos:NNS@[-1] & Word:.@[0]
 678 900 2221471  | NN->IN if Pos:VBD@[-1]
 640168610464192  | NN->IN if Pos

In [64]:
# http://www.nltk.org/api/nltk.tag.html?highlight=hmm#module-nltk.tag.crf
# CRF Tagger
ct = CRFTagger()
ct.train(brown_tagged_sents[0:500],'model.crf.tagger')
accuracy = ct.evaluate(brown_tagged_sents[0:500])

export_to_csv(ct.tag_sents([text])[0], 'data/crf_tagger.csv')
print("CRF Tagger Accuracy: {}".format(accuracy))

Exported Data to: data/crf_tagger.csv
CRF Tagger Accuracy: 0.9263085987533088


In [50]:
tags = [tag for (word, tag) in brown.tagged_words(categories='news')]
nltk.FreqDist(tags).max()

'NN'

In [51]:
tokens = nltk.word_tokenize(raw)
default_tagger = nltk.DefaultTagger('NN')
accuracy = default_tagger.evaluate(brown_tagged_sents)
export_to_csv(default_tagger.tag(text), 'data/default_tagger.csv')
print("Default Tagger Accuracy: {}".format(accuracy))

Exported Data to: data/default_tagger.csv
Default Tagger Accuracy: 0.13089484257215028


In [52]:
regexp_tagger = RegexpTagger([(r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
(r'(The|the|A|a|An|an)$', 'AT'),   # articles
(r'.*able$', 'JJ'),                # adjectives
(r'.*ness$', 'NN'),                # nouns formed from adjectives
(r'.*ly$', 'RB'),                  # adverbs
(r'.*s$', 'NNS'),                  # plural nouns
(r'.*ing$', 'VBG'),                # gerunds
(r'.*ed$', 'VBD'),                 # past tense verbs
(r'.*', 'NN')                      # nouns (default)
])
regexp_tagger.tag(brown_sents[3])
accuracy = regexp_tagger.evaluate(brown_tagged_sents)
export_to_csv(regexp_tagger.tag(text), 'data/regular_exp_tagger.csv')
print("REgular Expression Tagger Accuracy: {}".format(accuracy))

Exported Data to: data/regular_exp_tagger.csv
REgular Expression Tagger Accuracy: 0.304105256876902


In [53]:
fd = nltk.FreqDist(brown.words(categories='news'))
cfd = nltk.ConditionalFreqDist(brown.tagged_words(categories='news'))
most_freq_words = fd.most_common(100)
likely_tags = dict((word, cfd[word].max()) for (word, _) in most_freq_words)
baseline_tagger = nltk.UnigramTagger(model=likely_tags)
accuracy = baseline_tagger.evaluate(brown_tagged_sents)

export_to_csv(baseline_tagger.tag(text), 'data/lookup_tagger.csv')
print("Lookup Tagger Accuracy: {}".format(accuracy))

Exported Data to: data/lookup_tagger.csv
Lookup Tagger Accuracy: 0.45578495136941344


In [56]:
baseline_tagger.tag(text)

[('Lectures', None),
 ('were', 'BED'),
 ('really', None),
 ('good', None),
 ('.', '.'),
 ('There', None),
 ('were', 'BED'),
 ('lot', None),
 ('of', 'IN'),
 ('people', None),
 ('who', 'WPS'),
 ('came', None),
 ('their', 'PP$'),
 ('without', None),
 ('any', 'DTI'),
 ('Java', None),
 ('knowledge', None),
 ('and', 'CC'),
 ('yet', None),
 ('you', None),
 ('were', 'BED'),
 ('very', None),
 ('supportive', None),
 ('.', '.'),
 ('5/5', None),
 ('for', 'IN'),
 ('me', None),
 ('.', '.'),
 ('It', 'PPS'),
 ("'s", None),
 ('really', None),
 ('good', None),
 ('that', 'CS'),
 ('the', 'AT'),
 ('lecturer', None),
 ('explains', None),
 ('most', None),
 ('of', 'IN'),
 ('the', 'AT'),
 ('concepts', None),
 ('using', None),
 ('examples', None),
 ('.', '.'),
 ('It', 'PPS'),
 ('helps', None),
 ('us', None),
 ('to', 'TO'),
 ('understand', None),
 ('them', 'PPO'),
 ('better', None),
 ('.', '.'),
 ('But', 'CC'),
 ('if', None),
 ('you', None),
 ('are', 'BER'),
 ('sitting', None),
 ('at', 'IN'),
 ('the', 'AT'),
 ('

In [54]:
from nltk.corpus import brown
brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')
unigram_tagger = nltk.UnigramTagger(brown_tagged_sents)
accuracy = unigram_tagger.evaluate(brown_tagged_sents)
export_to_csv(unigram_tagger.tag(text), 'data/unigram_tagger.csv')
print("Unigram Tagger Accuracy: {}".format(accuracy))


Exported Data to: data/unigram_tagger.csv
Unigram Tagger Accuracy: 0.9349006503968017


In [55]:
unigram_tagger.tag(text)

[('Lectures', None),
 ('were', 'BED'),
 ('really', 'RB'),
 ('good', 'JJ'),
 ('.', '.'),
 ('There', 'EX'),
 ('were', 'BED'),
 ('lot', 'NN'),
 ('of', 'IN'),
 ('people', 'NNS'),
 ('who', 'WPS'),
 ('came', 'VBD'),
 ('their', 'PP$'),
 ('without', 'IN'),
 ('any', 'DTI'),
 ('Java', None),
 ('knowledge', 'NN'),
 ('and', 'CC'),
 ('yet', 'RB'),
 ('you', 'PPSS'),
 ('were', 'BED'),
 ('very', 'QL'),
 ('supportive', None),
 ('.', '.'),
 ('5/5', None),
 ('for', 'IN'),
 ('me', 'PPO'),
 ('.', '.'),
 ('It', 'PPS'),
 ("'s", None),
 ('really', 'RB'),
 ('good', 'JJ'),
 ('that', 'CS'),
 ('the', 'AT'),
 ('lecturer', 'NN'),
 ('explains', 'VBZ'),
 ('most', 'QL'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('concepts', None),
 ('using', 'VBG'),
 ('examples', None),
 ('.', '.'),
 ('It', 'PPS'),
 ('helps', 'VBZ'),
 ('us', 'PPO'),
 ('to', 'TO'),
 ('understand', 'VB'),
 ('them', 'PPO'),
 ('better', 'JJR'),
 ('.', '.'),
 ('But', 'CC'),
 ('if', 'CS'),
 ('you', 'PPSS'),
 ('are', 'BER'),
 ('sitting', 'VBG'),
 ('at', 'IN'),
 ('the